In [1]:
import pandas as pd
import numpy as np

In [2]:
educacion = pd.read_csv('../datos_preprocesados/fiuba_1_postulantes_educacion.csv')

In [3]:
educacion.head()

,idpostulante,nombre,estado
0,1d2B,Universitario,En Curso
1,6M3jr,Universitario,En Curso
2,NrlQx,Terciario/Técnico,En Curso
3,Y9RKV,Universitario,En Curso
4,ZL608,Universitario,En Curso


In [4]:
nivel_educativo = pd.get_dummies(educacion['nombre'])
educacion = pd.merge(nivel_educativo, educacion, left_index=True, right_index=True, how='right')

In [5]:
educacion.head()

,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,idpostulante,nombre,estado
0,0,0,0,0,0,0,1,1d2B,Universitario,En Curso
1,0,0,0,0,0,0,1,6M3jr,Universitario,En Curso
2,0,0,0,0,0,1,0,NrlQx,Terciario/Técnico,En Curso
3,0,0,0,0,0,0,1,Y9RKV,Universitario,En Curso
4,0,0,0,0,0,0,1,ZL608,Universitario,En Curso


In [6]:
def set_estado(reg):
    multiplicador = {'En Curso': 1, 'Abandonado': 2, 'Graduado': 3}[reg['estado']]
    reg[reg['nombre']] = reg[reg['nombre']] * multiplicador
    return reg

educacion = educacion.apply(set_estado, axis=1)
educacion = educacion.drop(axis=1, labels=['nombre', 'estado'])
educacion = educacion.groupby('idpostulante').agg(max)

In [7]:
educacion = educacion.reset_index()
educacion.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,0z5Dmrd,0,0,0,0,0,0,3
1,0z5JW1r,0,0,0,0,0,0,2
2,0z5VvGv,0,0,0,0,3,0,0
3,0zB01pE,0,0,0,0,0,0,2
4,0zB026d,0,0,0,0,0,3,0


In [8]:
genero_edad = pd.read_csv('../datos_preprocesados/fiuba_2_postulantes_genero_y_edad.csv')

In [9]:
genero_edad.head()

,idpostulante,fechanacimiento,sexo
0,eo2p,1981-02-16,MASC
1,1d2B,1976-02-28,MASC
2,EBO0,1973-09-11,FEM
3,a6MKW,1974-06-07,MASC
4,6MWd4,1974-12-19,MASC


In [10]:
genero_edad = genero_edad[(genero_edad['sexo'] != 'NO_DECLARA') |  (genero_edad['fechanacimiento'].isna() == False)]

In [11]:
postulantes = pd.merge(educacion, genero_edad, on='idpostulante', how='inner')

In [12]:
postulantes.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,0z5Dmrd,0,0,0,0,0,0,3,1965-10-22,MASC
1,0z5JW1r,0,0,0,0,0,0,2,1971-04-09,MASC
2,0z5VvGv,0,0,0,0,3,0,0,1987-10-18,MASC
3,0zB01pE,0,0,0,0,0,0,2,1989-12-01,MASC
4,0zB026d,0,0,0,0,0,3,0,1978-02-23,FEM


In [13]:
for columna in ['Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario', 'Terciario/Técnico', 'Universitario']:
    postulantes[columna] = postulantes[columna].astype('int')
    postulantes[columna] = postulantes[columna].map(lambda x: {1: 'En Curso', 3: 'Graduado', 2: 'Abandonado', 0:'No'}[x])
    postulantes[columna] = postulantes[columna].astype('category')

In [14]:
postulantes['sexo'] = postulantes['sexo'].astype('category')

In [15]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440160 entries, 0 to 440159
Data columns (total 10 columns):
idpostulante         440160 non-null object
Doctorado            440160 non-null category
Master               440160 non-null category
Otro                 440160 non-null category
Posgrado             440160 non-null category
Secundario           440160 non-null category
Terciario/Técnico    440160 non-null category
Universitario        440160 non-null category
fechanacimiento      438730 non-null object
sexo                 440160 non-null category
dtypes: category(8), object(2)
memory usage: 13.4+ MB


In [16]:
postulantes['fechanacimiento'] = pd.to_datetime(postulantes['fechanacimiento'], errors = 'coerce', format= '%Y-%m-%d')


In [17]:
postulantes = postulantes[~(postulantes['fechanacimiento'].isnull())]

In [18]:
from datetime import date
today = date.today()
fechas = postulantes['fechanacimiento']
postulantes['edad'] = fechas.map(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)))
postulantes = postulantes.loc[~( (postulantes['edad'] < 16) | (postulantes['edad'] > 75) )]
postulantes = postulantes.drop(labels='fechanacimiento', axis=1)
postulantes.head()


,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,sexo,edad
0,0z5Dmrd,No,No,No,No,No,No,Graduado,MASC,52
1,0z5JW1r,No,No,No,No,No,No,Abandonado,MASC,47
2,0z5VvGv,No,No,No,No,Graduado,No,No,MASC,30
3,0zB01pE,No,No,No,No,No,No,Abandonado,MASC,28
4,0zB026d,No,No,No,No,No,Graduado,No,FEM,40


In [19]:
postulantes.to_csv('../datos_preprocesados/postulantes.csv', index=False)

In [2]:
vistas = pd.read_csv('../datos_preprocesados/fiuba_3_vistas.csv')
vistas.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [3]:
vistas = vistas[['idAviso','idpostulante']]

In [70]:
avisos_detalle = pd.read_csv('../datos_preprocesados/fiuba_6_avisos_detalle.csv')
avisos_detalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1001284385,1,Dejanos tu CV,<p><br />Si est&aacute;s interesado en ser par...,Corrientes,NaN,NaN,Full-time,NaN,Otros,Establecimiento Las Marías
1,1110092955,1,para línea de producción (ref. 28376),<p>Empresas vinculadas a la industria automotr...,GBA Oeste,NaN,NaN,Full-time,NaN,Producción,RANDSTAD
2,1110092958,1,para mantenimiento (ref. 27181),"<p style=""display: inline !important;"">Empresa...",GBA Oeste,NaN,NaN,Full-time,NaN,Mantenimiento,RANDSTAD
3,1110391074,1,Trabajá con nosotros,<p><strong>Trabaj&aacute; con nosotros</strong...,Cordoba,NaN,NaN,Full-time,NaN,Ingeniería Industrial,Coca Cola Andina Argentina
4,1110669632,1,Técnico de Mantenimiento,<p>Experiencia en f&aacute;bricas cosm&eacute;...,GBA Oeste,NaN,", Lanús",Full-time,NaN,Mantenimiento,PLUMARI S A


In [71]:
avisos_detalle = avisos_detalle[['idaviso', 'nombre_area']]
avisos_detalle.head()

,idaviso,nombre_area
0,1001284385,Otros
1,1110092955,Producción
2,1110092958,Mantenimiento
3,1110391074,Ingeniería Industrial
4,1110669632,Mantenimiento


In [6]:
vistas = pd.merge(vistas, avisos_detalle, left_on='idAviso', right_on='idaviso')
vistas = vistas.drop(labels='idaviso', axis=1)
vistas.head()

,idAviso,idpostulante,nombre_area
0,1111780242,YjVJQ6Z,Programación de producción
1,1111780242,dYjRpMD,Programación de producción
2,1111780242,wVopPXl,Programación de producción
3,1111780242,wVopPXl,Programación de producción
4,1111780242,wVoOPoL,Programación de producción


In [7]:
def max_cat(x):
    return (x).value_counts().index[0]
def max_value_cat(x):
    return x.value_counts().values[0]

In [8]:
grouped = vistas.groupby('idpostulante')
vistas2 = grouped.aggregate([max_cat, max_value_cat])
vistas2.head()

idAviso                          nombre_area              
              max_cat max_value_cat                max_cat max_value_cat
idpostulante                                                            
0005E         1785430             2  Tecnologia / Sistemas             2
00Lkv         1790030             2      Comercio Exterior             2
00dMd         1790030             4      Comercio Exterior             4
01QAq         1803890             1           Contabilidad             1
021OM         1790030             5      Comercio Exterior             5

In [10]:
vistas2 = vistas2.reset_index()
vistas2.tail()

index idpostulante  idAviso                          nombre_area  \
                             max_cat max_value_cat                max_cat   
400184  400184        zxk4k  1799580             2              Servicios   
400185  400185        zxqXm  1803810             1  Tecnologia / Sistemas   
400186  400186        zzPXE  1790030             5      Comercio Exterior   
400187  400187        zzQjm  1790030             4      Comercio Exterior   
400188  400188        zzmMm  1790030             1      Comercio Exterior   

                      
       max_value_cat  
400184             2  
400185             1  
400186             5  
400187             4  
400188             1

In [11]:
vistas2.to_csv('../datos_preprocesados/max_aviso_y_cat_con_valor.csv', index=False)

In [14]:
distintos = vistas2[vistas2[('idAviso','max_value_cat')] != vistas2[('nombre_area','max_value_cat')]]
distintos.shape

(231367, 6)

In [19]:
def second_cat(x):
    try: 
        return (x).value_counts().index[1]
    except IndexError:
        return None
def second_value_cat(x):
    try:
        return x.value_counts().values[1]
    except IndexError:
        return 0

In [20]:
vistas3 = grouped.aggregate([second_cat, second_value_cat])
vistas3.head()

idAviso                  nombre_area                 
             second_cat second_value_cat  second_cat second_value_cat
idpostulante                                                         
0005E               NaN                0        None                0
00Lkv               NaN                0        None                0
00dMd               NaN                0        None                0
01QAq               NaN                0        None                0
021OM               NaN                0        None                0

In [21]:
vistas3.tail()

idAviso                    nombre_area                 
             second_cat second_value_cat    second_cat second_value_cat
idpostulante                                                           
zxk4k               NaN                0          None                0
zxqXm         1790030.0                1  Contabilidad                1
zzPXE               NaN                0          None                0
zzQjm               NaN                0          None                0
zzmMm               NaN                0          None                0

In [22]:
vistas3.to_csv('../datos_preprocesados/second_aviso_y_cat_con_valor.csv', index=False)

In [23]:
vistas3[('idAviso','second_cat')].isnull().sum()

95249

In [24]:
vistas3.shape

(400189, 4)

In [31]:
vistas3 = vistas3.reset_index()

In [26]:
def third_cat(x):
    try: 
        return (x).value_counts().index[2]
    except IndexError:
        return None
def third_value_cat(x):
    try:
        return x.value_counts().values[2]
    except IndexError:
        return 0

In [27]:
vistas4 = grouped.aggregate([third_cat, third_value_cat])
vistas4.head()

idAviso                 nombre_area                
             third_cat third_value_cat   third_cat third_value_cat
idpostulante                                                      
0005E              NaN               0        None               0
00Lkv              NaN               0        None               0
00dMd              NaN               0        None               0
01QAq              NaN               0        None               0
021OM              NaN               0        None               0

In [29]:
vistas4.to_csv('../datos_preprocesados/third_aviso_y_cat_con_valor.csv', index=False)

In [30]:
vistas4[('idAviso','third_cat')].isnull().sum()

139769

In [32]:
vistas4 = vistas4.reset_index()

In [86]:
vistas = pd.merge(vistas2,vistas3, on='idpostulante')
vistas.head()

C:\Users\Franco\Anaconda3\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


index idpostulante  idAviso                          nombre_area  \
                      max_cat max_value_cat                max_cat   
0     0        0005E  1785430             2  Tecnologia / Sistemas   
1     1        00Lkv  1790030             2      Comercio Exterior   
2     2        00dMd  1790030             4      Comercio Exterior   
3     3        01QAq  1803890             1           Contabilidad   
4     4        021OM  1790030             5      Comercio Exterior   

                   idAviso                  nombre_area                   
  max_value_cat second_cat second_value_cat  second_cat second_value_cat  
0             2        NaN                0        None                0  
1             2        NaN                0        None                0  
2             4        NaN                0        None                0  
3             1        NaN                0        None                0  
4             5        NaN                0        None                0

In [87]:
vistas = pd.merge(vistas,vistas4, on='idpostulante')

C:\Users\Franco\Anaconda3\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [88]:
vistas.head()

index idpostulante  idAviso                          nombre_area  \
                      max_cat max_value_cat                max_cat   
0     0        0005E  1785430             2  Tecnologia / Sistemas   
1     1        00Lkv  1790030             2      Comercio Exterior   
2     2        00dMd  1790030             4      Comercio Exterior   
3     3        01QAq  1803890             1           Contabilidad   
4     4        021OM  1790030             5      Comercio Exterior   

                   idAviso                  nombre_area                   \
  max_value_cat second_cat second_value_cat  second_cat second_value_cat   
0             2        NaN                0        None                0   
1             2        NaN                0        None                0   
2             4        NaN                0        None                0   
3             1        NaN                0        None                0   
4             5        NaN                0        None                0   

    idAviso                 nombre_area                  
  third_cat third_value_cat   third_cat third_value_cat  
0       NaN               0        None               0  
1       NaN               0        None               0  
2       NaN               0        None               0  
3       NaN               0        None               0  
4       NaN               0        None               0

In [89]:
vistas = vistas.drop(labels='index', axis=1)
vistas.head()

C:\Users\Franco\Anaconda3\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


idpostulante  idAviso                          nombre_area                \
                max_cat max_value_cat                max_cat max_value_cat   
0        0005E  1785430             2  Tecnologia / Sistemas             2   
1        00Lkv  1790030             2      Comercio Exterior             2   
2        00dMd  1790030             4      Comercio Exterior             4   
3        01QAq  1803890             1           Contabilidad             1   
4        021OM  1790030             5      Comercio Exterior             5   

     idAviso                  nombre_area                    idAviso  \
  second_cat second_value_cat  second_cat second_value_cat third_cat   
0        NaN                0        None                0       NaN   
1        NaN                0        None                0       NaN   
2        NaN                0        None                0       NaN   
3        NaN                0        None                0       NaN   
4        NaN                0        None                0       NaN   

                  nombre_area                  
  third_value_cat   third_cat third_value_cat  
0               0        None               0  
1               0        None               0  
2               0        None               0  
3               0        None               0  
4               0        None               0

In [90]:
vistas.columns = ['_'.join(col) for col in vistas.columns]
vistas.head()

,idpostulante_,idAviso_max_cat,idAviso_max_value_cat,nombre_area_max_cat,nombre_area_max_value_cat,idAviso_second_cat,idAviso_second_value_cat,nombre_area_second_cat,nombre_area_second_value_cat,idAviso_third_cat,idAviso_third_value_cat,nombre_area_third_cat,nombre_area_third_value_cat
0,0005E,1785430,2,Tecnologia / Sistemas,2,NaN,0,None,0,NaN,0,None,0
1,00Lkv,1790030,2,Comercio Exterior,2,NaN,0,None,0,NaN,0,None,0
2,00dMd,1790030,4,Comercio Exterior,4,NaN,0,None,0,NaN,0,None,0
3,01QAq,1803890,1,Contabilidad,1,NaN,0,None,0,NaN,0,None,0
4,021OM,1790030,5,Comercio Exterior,5,NaN,0,None,0,NaN,0,None,0


In [91]:
vistas = vistas.rename(columns={'idpostulante_':'idpostulante', 'idAviso_max_cat':'1_idaviso', 'idAviso_max_value_cat' : 'cant_1_aviso', 'idAviso_second_cat':'2_idaviso', 'idAviso_second_value_cat' : 'cant_2_aviso', 'idAviso_third_cat':'3_idaviso', 'idAviso_third_value_cat' : 'cant_3_aviso'})
vistas.head()

,idpostulante,1_idaviso,cant_1_aviso,nombre_area_max_cat,nombre_area_max_value_cat,2_idaviso,cant_2_aviso,nombre_area_second_cat,nombre_area_second_value_cat,3_idaviso,cant_3_aviso,nombre_area_third_cat,nombre_area_third_value_cat
0,0005E,1785430,2,Tecnologia / Sistemas,2,NaN,0,None,0,NaN,0,None,0
1,00Lkv,1790030,2,Comercio Exterior,2,NaN,0,None,0,NaN,0,None,0
2,00dMd,1790030,4,Comercio Exterior,4,NaN,0,None,0,NaN,0,None,0
3,01QAq,1803890,1,Contabilidad,1,NaN,0,None,0,NaN,0,None,0
4,021OM,1790030,5,Comercio Exterior,5,NaN,0,None,0,NaN,0,None,0


In [92]:
vistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400189 entries, 0 to 400188
Data columns (total 13 columns):
idpostulante                    400189 non-null object
1_idaviso                       400189 non-null int64
cant_1_aviso                    400189 non-null int64
nombre_area_max_cat             400189 non-null object
nombre_area_max_value_cat       400189 non-null int64
2_idaviso                       304940 non-null float64
cant_2_aviso                    400189 non-null int64
nombre_area_second_cat          279983 non-null object
nombre_area_second_value_cat    400189 non-null int64
3_idaviso                       260420 non-null float64
cant_3_aviso                    400189 non-null int64
nombre_area_third_cat           219863 non-null object
nombre_area_third_value_cat     400189 non-null int64
dtypes: float64(2), int64(7), object(4)
memory usage: 42.7+ MB


In [52]:
vistas['2_idaviso'] = vistas['2_idaviso'].fillna(0)
vistas['3_idaviso'] = vistas['3_idaviso'].fillna(0)
vistas['nombre_area_second_cat'] = vistas['nombre_area_second_cat'].fillna('Ninguna')
vistas['nombre_area_third_cat'] = vistas['nombre_area_third_cat'].fillna('Ninguna')
vistas.head()

,idpostulante,1_idaviso,cant_1_aviso,nombre_area_max_cat,nombre_area_max_value_cat,2_idaviso,cant_2_aviso,nombre_area_second_cat,nombre_area_second_value_cat,3_idaviso,cant_3_aviso,nombre_area_third_cat,nombre_area_third_value_cat
0,0005E,1785430,2,Tecnologia / Sistemas,2,0.0,0,Ninguna,0,0.0,0,Ninguna,0
1,00Lkv,1790030,2,Comercio Exterior,2,0.0,0,Ninguna,0,0.0,0,Ninguna,0
2,00dMd,1790030,4,Comercio Exterior,4,0.0,0,Ninguna,0,0.0,0,Ninguna,0
3,01QAq,1803890,1,Contabilidad,1,0.0,0,Ninguna,0,0.0,0,Ninguna,0
4,021OM,1790030,5,Comercio Exterior,5,0.0,0,Ninguna,0,0.0,0,Ninguna,0


In [53]:
vistas.to_csv('../datos_preprocesados/1_to_3_avisos_y_cant_por_postulante.csv', index=False)

In [57]:
vistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400189 entries, 0 to 400188
Data columns (total 13 columns):
idpostulante                    400189 non-null object
1_idaviso                       400189 non-null int64
cant_1_aviso                    400189 non-null int64
nombre_area_max_cat             400189 non-null object
nombre_area_max_value_cat       400189 non-null int64
2_idaviso                       400189 non-null float64
cant_2_aviso                    400189 non-null int64
nombre_area_second_cat          400189 non-null object
nombre_area_second_value_cat    400189 non-null int64
3_idaviso                       400189 non-null float64
cant_3_aviso                    400189 non-null int64
nombre_area_third_cat           400189 non-null object
nombre_area_third_value_cat     400189 non-null int64
dtypes: float64(2), int64(7), object(4)
memory usage: 42.7+ MB


In [2]:
import pandas as pd
import numpy as np

In [3]:
vistas = pd.read_csv('../datos_preprocesados/1_to_3_avisos_y_cant_por_postulante.csv')
postulantes = pd.read_csv('../datos_preprocesados/postulantes.csv')
postulantes.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,sexo,edad
0,0z5Dmrd,No,No,No,No,No,No,Graduado,MASC,52
1,0z5JW1r,No,No,No,No,No,No,Abandonado,MASC,47
2,0z5VvGv,No,No,No,No,Graduado,No,No,MASC,30
3,0zB01pE,No,No,No,No,No,No,Abandonado,MASC,28
4,0zB026d,No,No,No,No,No,Graduado,No,FEM,40


In [4]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438678 entries, 0 to 438677
Data columns (total 10 columns):
idpostulante         438678 non-null object
Doctorado            438678 non-null object
Master               438678 non-null object
Otro                 438678 non-null object
Posgrado             438678 non-null object
Secundario           438678 non-null object
Terciario/Técnico    438678 non-null object
Universitario        438678 non-null object
sexo                 438678 non-null object
edad                 438678 non-null int64
dtypes: int64(1), object(9)
memory usage: 33.5+ MB


In [5]:
postulantes = pd.merge(postulantes, vistas, on='idpostulante')
postulantes.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,sexo,edad,...,nombre_area_max_cat,nombre_area_max_value_cat,2_idaviso,cant_2_aviso,nombre_area_second_cat,nombre_area_second_value_cat,3_idaviso,cant_3_aviso,nombre_area_third_cat,nombre_area_third_value_cat
0,0z5Dmrd,No,No,No,No,No,No,Graduado,MASC,52,...,Recursos Humanos,5,1.112384e+09,1,Ninguna,0,1.112420e+09,1,Ninguna,0
1,0z5JW1r,No,No,No,No,No,No,Abandonado,MASC,47,...,Gerencia / Dirección General,5,1.112417e+09,3,Operaciones,4,1.112451e+09,2,Ventas,4
2,0z5VvGv,No,No,No,No,Graduado,No,No,MASC,30,...,Atención al Cliente,2,1.112406e+09,1,Call Center,1,1.112225e+09,1,Administración,1
3,0zB01pE,No,No,No,No,No,No,Abandonado,MASC,28,...,Administracion de Seguros,2,1.112442e+09,1,Comercial,1,1.112424e+09,1,Contabilidad,1
4,0zB026d,No,No,No,No,No,Graduado,No,FEM,40,...,Mantenimiento,10,0.000000e+00,0,Ninguna,0,0.000000e+00,0,Ninguna,0


In [6]:
postulantes.shape

(334470, 22)

In [7]:
postulaciones = pd.read_csv('../datos_preprocesados/fiuba_4_postulaciones.csv')
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [8]:
postulaciones.shape

(6604534, 3)

In [9]:
postulaciones = postulaciones[['idaviso','idpostulante']]

In [10]:
postulaciones = pd.merge(postulaciones, postulantes, on='idpostulante')
postulaciones.head()

,idaviso,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,sexo,...,nombre_area_max_cat,nombre_area_max_value_cat,2_idaviso,cant_2_aviso,nombre_area_second_cat,nombre_area_second_value_cat,3_idaviso,cant_3_aviso,nombre_area_third_cat,nombre_area_third_value_cat
0,1112257047,NM5M,No,No,No,No,Graduado,No,No,FEM,...,Atención al Cliente,4,1.112444e+09,2,Telemarketing,2,0.000000e+00,0,Ninguna,0
1,1111920714,NM5M,No,No,No,No,Graduado,No,No,FEM,...,Atención al Cliente,4,1.112444e+09,2,Telemarketing,2,0.000000e+00,0,Ninguna,0
2,1112346945,NM5M,No,No,No,No,Graduado,No,No,FEM,...,Atención al Cliente,4,1.112444e+09,2,Telemarketing,2,0.000000e+00,0,Ninguna,0
3,1112345547,NM5M,No,No,No,No,Graduado,No,No,FEM,...,Atención al Cliente,4,1.112444e+09,2,Telemarketing,2,0.000000e+00,0,Ninguna,0
4,1112237522,5awk,No,No,No,No,No,Graduado,No,FEM,...,Contabilidad,26,1.112315e+09,3,Impuestos,3,1.112387e+09,3,Medicina,3


In [11]:
postulaciones.shape

(5200998, 23)

In [12]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5200998 entries, 0 to 5200997
Data columns (total 23 columns):
idaviso                         int64
idpostulante                    object
Doctorado                       object
Master                          object
Otro                            object
Posgrado                        object
Secundario                      object
Terciario/Técnico               object
Universitario                   object
sexo                            object
edad                            int64
1_idaviso                       int64
cant_1_aviso                    int64
nombre_area_max_cat             object
nombre_area_max_value_cat       int64
2_idaviso                       float64
cant_2_aviso                    int64
nombre_area_second_cat          object
nombre_area_second_value_cat    int64
3_idaviso                       float64
cant_3_aviso                    int64
nombre_area_third_cat           object
nombre_area_third_value_cat     int

In [13]:
postulaciones['Doctorado'] = postulaciones['Doctorado'].astype('category')
postulaciones['Master'] = postulaciones['Master'].astype('category')
postulaciones['Otro'] = postulaciones['Otro'].astype('category')
postulaciones['Posgrado'] = postulaciones['Posgrado'].astype('category')
postulaciones['Secundario'] = postulaciones['Secundario'].astype('category')
postulaciones['Terciario/Técnico'] = postulaciones['Terciario/Técnico'].astype('category')
postulaciones['Universitario'] = postulaciones['Universitario'].astype('category')
postulaciones['sexo'] = postulaciones['sexo'].astype('category')
postulaciones['nombre_area_max_cat'] = postulaciones['nombre_area_max_cat'].astype('category')
postulaciones['nombre_area_second_cat'] = postulaciones['nombre_area_second_cat'].astype('category')
postulaciones['nombre_area_third_cat'] = postulaciones['nombre_area_third_cat'].astype('category')
postulaciones['2_idaviso'] = postulaciones['2_idaviso'].astype('int64')
postulaciones['3_idaviso'] = postulaciones['3_idaviso'].astype('int64')

In [94]:
postulaciones.to_csv('../datos_preprocesados/vistas_y_postulaciones_por_aviso.csv', index=False)

In [14]:
#postulaciones = pd.read_csv('../datos_preprocesados/vistas_y_postulaciones_por_aviso.csv')
postulaciones = postulaciones.sample(n=100000,axis=0)
resultados = postulaciones['idaviso']
postulaciones = postulaciones.drop(labels = 'idaviso', axis= 1)
postulaciones = postulaciones.drop(labels = 'idpostulante', axis= 1)

In [15]:
resultados.head()

4105327    1112382017
1907780    1112053431
531388     1112281847
3473331    1112399812
1604969    1112323062
Name: idaviso, dtype: int64

In [16]:
postulaciones.head()

,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,sexo,edad,1_idaviso,...,nombre_area_max_cat,nombre_area_max_value_cat,2_idaviso,cant_2_aviso,nombre_area_second_cat,nombre_area_second_value_cat,3_idaviso,cant_3_aviso,nombre_area_third_cat,nombre_area_third_value_cat
4105327,No,No,No,Graduado,No,No,No,FEM,44,1112384412,...,Planeamiento económico-financiero,5,1111043912,3,Finanzas,4,1112402536,2,Secretaria,3
1907780,No,No,No,No,Graduado,No,No,FEM,21,1112316934,...,Ventas,61,1112358408,5,Mantenimiento y Limpieza,21,1112325383,4,Gastronomia,10
531388,No,No,No,No,No,Graduado,No,FEM,21,1111662226,...,Farmacéutica,1,0,0,Ninguna,0,0,0,Ninguna,0
3473331,No,No,No,No,No,Graduado,No,MASC,64,1112176012,...,Seguridad e Higiene,88,1112409835,18,Producción,34,1112355043,17,Ingeniería Industrial,9
1604969,No,Graduado,No,No,No,No,No,MASC,34,1112446621,...,Legal,12,1112414236,2,Recursos Humanos,7,1112408215,2,Ninguna,0


In [17]:
from sklearn import tree
from sklearn import preprocessing as pre
le = pre.LabelEncoder()
le.fit(postulaciones['sexo'])
postulaciones['sexo']= le.transform(postulaciones['sexo'])
le.fit(postulaciones['Doctorado'])
postulaciones['Doctorado']= le.transform(postulaciones['Doctorado']) 
le.fit(postulaciones['Master'])
postulaciones['Master']= le.transform(postulaciones['Master'])
le.fit(postulaciones['Otro'])
postulaciones['Otro']= le.transform(postulaciones['Otro'])
le.fit(postulaciones['Posgrado'])
postulaciones['Posgrado']= le.transform(postulaciones['Posgrado'])
le.fit(postulaciones['Secundario'])
postulaciones['Secundario']= le.transform(postulaciones['Secundario'])
le.fit(postulaciones['Terciario/Técnico'])
postulaciones['Terciario/Técnico']= le.transform(postulaciones['Terciario/Técnico'])
le.fit(postulaciones['Universitario'])
postulaciones['Universitario']= le.transform(postulaciones['Universitario'])
le.fit(postulaciones['nombre_area_max_cat'])
postulaciones['nombre_area_max_cat']= le.transform(postulaciones['nombre_area_max_cat'])
le.fit(postulaciones['nombre_area_second_cat'])
postulaciones['nombre_area_second_cat']= le.transform(postulaciones['nombre_area_second_cat'])
le.fit(postulaciones['nombre_area_third_cat'])
postulaciones['nombre_area_third_cat']= le.transform(postulaciones['nombre_area_third_cat'])
postulaciones.head()

,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,sexo,edad,1_idaviso,...,nombre_area_max_cat,nombre_area_max_value_cat,2_idaviso,cant_2_aviso,nombre_area_second_cat,nombre_area_second_value_cat,3_idaviso,cant_3_aviso,nombre_area_third_cat,nombre_area_third_value_cat
4105327,3,3,3,2,3,3,3,0,44,1112384412,...,123,5,1111043912,3,72,4,1112402536,2,148,3
1907780,3,3,3,3,2,3,3,0,21,1112316934,...,161,61,1112358408,5,107,21,1112325383,4,77,10
531388,3,3,3,3,3,2,3,0,21,1111662226,...,69,1,0,0,115,0,0,0,121,0
3473331,3,3,3,3,3,2,3,1,64,1112176012,...,143,88,1112409835,18,128,34,1112355043,17,85,9
1604969,3,2,3,3,3,3,3,1,34,1112446621,...,101,12,1112414236,2,138,7,1112408215,2,121,0


In [20]:
entrenamiento = postulaciones[:50000]
resultados_entrenamiento = resultados[:50000]
prueba = postulaciones[50000:60000]
resultados_prueba = resultados[50000:60000]

In [28]:
clf = tree.DecisionTreeClassifier(min_samples_split= 20, max_depth=25)
clf = clf.fit(entrenamiento.values, resultados_entrenamiento.values)

In [29]:
clf.score(prueba.values,resultados_prueba.values)

0.0031

In [31]:
prediccion = clf.predict_proba([prueba.values[1]])
prediccion.shape

(1, 12045)

In [26]:
prueba

,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,sexo,edad,1_idaviso,...,nombre_area_max_cat,nombre_area_max_value_cat,2_idaviso,cant_2_aviso,nombre_area_second_cat,nombre_area_second_value_cat,3_idaviso,cant_3_aviso,nombre_area_third_cat,nombre_area_third_value_cat
166437,3,3,3,3,2,3,3,0,19,1112351629,...,133,5,1112349336,2,143,5,1112335122,2,121,0
3241832,3,3,3,3,3,3,1,1,31,1112487295,...,2,32,1112418679,3,159,17,1112483962,3,75,8
3499234,3,3,3,3,2,3,3,1,39,1112206678,...,161,50,1111563021,6,27,36,1111529625,6,136,31
122289,3,3,3,3,1,3,3,1,29,1112363352,...,124,10,1112320301,5,59,7,1112299445,2,168,2
2114180,3,3,3,1,3,3,3,0,34,1112350306,...,151,1,0,0,115,0,0,0,121,0
4029540,3,3,3,3,3,3,0,0,31,1112420942,...,11,5,1112450843,2,137,4,1112432810,2,148,2
2552190,3,3,3,3,2,3,3,1,30,1112428743,...,37,6,1112363005,1,2,4,1112365114,1,45,1
3346513,3,3,3,3,2,3,3,1,25,1112413223,...,2,12,1112372699,6,105,12,1112433181,6,171,10
4742855,3,3,3,3,2,3,3,0,21,1112167719,...,2,18,1112426801,4,123,9,1112455276,4,142,8
2230389,3,3,3,3,2,3,3,0,24,1112436468,...,161,73,1112467026,5,13,19,1112447911,4,164,12
